In [2]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

import module as module

In [3]:
#fichier données
data = './Data/Source/'
app_train = pd.read_csv(data+'application_train.csv')
app_test = pd.read_csv(data+'application_test.csv')

In [4]:
module.repartitionCibles(app_train)

Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)


In [132]:
X_train, X_test, labels, features = module.traitementDonnées(app_train, app_test, smote=False)

----------- Datas before traitments ----------
Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)
..............................................
Training data shape:  (307511, 130)
Testing data shape:  (48744, 129)
----------- Datas after traitments -----------
Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)
..............................................
Training data shape:  (307511, 260)
Testing data shape:  (48744, 260)
------------- Time of traitments -------------
Time: 10.8s


In [133]:
X_train, X_test, labels, features = module.traitementDonnées(app_train, app_test, smote=True)

----------- Datas before traitments ----------
Ratio des valeurs cibles
valeurs positives 8.1% (24825)
valeurs négatives 91.9% (282686)
..............................................
Training data shape:  (307511, 130)
Testing data shape:  (48744, 129)
------------- application SMOTE --------------
----------- Datas after traitments -----------
Ratio des valeurs cibles
valeurs positives 50.0% (282686)
valeurs négatives 50.0% (282686)
..............................................
Training data shape:  (565372, 260)
Testing data shape:  (48744, 260)
------------- Time of traitments -------------
Time: 14.2s


In [142]:
from sklearn.linear_model import LogisticRegression
import mlflow
mlflow.set_tracking_uri(uri="http://10.0.50.72:5000")
mlflow.set_experiment(experiment_id="922754860405524204")
mlflow.set_tag('mlflow.runName', 'LogisticRegression')
mlflow.autolog()

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(X_train, labels)
log_reg_train = log_reg.predict_proba(X_train)[:, 1]

2024/12/31 19:29:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/12/31 19:29:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2024/12/31 19:29:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


In [149]:
log_reg_pred = log_reg.predict_proba(X_test)[:, 1]

In [130]:
import importlib 
importlib.reload(module)


<module 'module' from 'Z:\\P7\\module.py'>

In [ ]:
mlflow.set_tag('mlflow.runName', 'new-run-name')